# Delta Lake Managed vs External Tables

This notebook demonstrates how to create managed and external tables with Delta Lake.

In [1]:
import pyspark
from delta import *
from pyspark.sql.types import *

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [3]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ceb609e5-3b8e-460a-a766-9895069f0e29;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 317ms :: artifacts dl 17ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number|

22/09/12 10:09:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/12 10:09:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/12 10:09:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Create external Delta Lake table with save

In [4]:
columns = ["movie", "release_date"]
data = [("The Godfather", 1972), ("Detective Pikachu", 2019), ("Donny Darko", 2001)]
rdd = spark.sparkContext.parallelize(data)
df = rdd.toDF(columns)

In [5]:
df.write.format("delta").save("external_table1")

In [6]:
spark.read.format("delta").load("external_table1").show()

+-----------------+------------+
|            movie|release_date|
+-----------------+------------+
|Detective Pikachu|        2019|
|    The Godfather|        1972|
|      Donny Darko|        2001|
+-----------------+------------+



In [12]:
df.createOrReplaceTempView("some_external_table")

In [13]:
spark.sql("DESCRIBE TABLE EXTENDED some_external_table").show(truncate=False)

+------------+---------+-------+
|col_name    |data_type|comment|
+------------+---------+-------+
|movie       |string   |null   |
|release_date|bigint   |null   |
+------------+---------+-------+

22/09/12 10:25:40 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /private/var/folders/19/_52w4zps3xjc6plz_f63j8sh0000gp/T/spark-e20f8ea5-85f5-4630-b664-28f64608a43b. Falling back to Java IO way
java.io.IOException: Failed to delete: /private/var/folders/19/_52w4zps3xjc6plz_f63j8sh0000gp/T/spark-e20f8ea5-85f5-4630-b664-28f64608a43b
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:171)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:110)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:91)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1206)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4(ShutdownHookManager.scala:65)
	at org.apach

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53678)
Traceback (most recent call last):
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/site-packages/pyspark/accumulators.py", line 281, in ha

In [7]:
%rm -rf external_table1

## Create external Delta Lake table with saveAsTable

In [8]:
df.write.format("delta").option("path", "external_table2").saveAsTable(
    "default.external_table2"
)

In [10]:
spark.sql("select * from external_table2").show()

AnalysisException: `default`.`external_table2` is not a Delta table.

In [15]:
%rm -rf external_table2

## Creating Delta Lake Managed Table

In [7]:
df.write.format("delta").saveAsTable("some_managed_table")

In [8]:
spark.sql("select * from some_managed_table").show()

+-----------------+------------+
|            movie|release_date|
+-----------------+------------+
|Detective Pikachu|        2019|
|    The Godfather|        1972|
|      Donny Darko|        2001|
+-----------------+------------+



In [11]:
spark.sql("DESCRIBE TABLE EXTENDED some_managed_table").show(truncate=False)

+----------------------------+---------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                            |comment|
+----------------------------+---------------------------------------------------------------------------------------------------------------------+-------+
|movie                       |string                                                                                                               |       |
|release_date                |bigint                                                                                                               |       |
|                            |                                                                                                                     |       |
|# Partitioning              |                            